In [2]:
import numpy as np
import sympy as smp
from sympy import *
from IPython.display import display, Latex

In [3]:
G, M, t, r, theta, phi, k, c = smp.symbols('G M t r θ Φ k c')
a = smp.Function('a')
a_dot = smp.Function('ȧ')
a_Dot = smp.Function('ä')

Gamma = IndexedBase("Gamma")
Riemann = IndexedBase("R")

In [4]:
Var = {
    t : 0,
    r : 1,
    theta : 2,
    phi : 3
}

In [5]:
Rav = {
    0 : t,
    1 : r,
    2 : theta,
    3 : phi
}

# Computing Ricci Tensor

### $$R_{\mu \nu} = R^{\alpha}_{\mu \alpha \nu}$$

In [6]:
# Compute Christoffel Symbols
def Christoffel_Symbols(m, i, j, Metric):
    inv_Metric = Metric.inv()
    C = 0
    
    for l in range(4):
        #print(1/2*inv_Metric[Var[m], l])
        C += 1/2*inv_Metric[Var[m], l]*(smp.diff(Metric[Var[i],Var[m]], j)+smp.diff(Metric[Var[m],Var[j]], i)-smp.diff(Metric[Var[j],Var[i]], m))
    return simplify(C)

In [7]:
# Compute all Christoffel Symbols
def All_Christoffel_Symbols(Metric):
    All_C = np.zeros((4,4,4), dtype=object)
    
    for k in range(4):
        for i in range(4):
            for j in range(4):
                c = Christoffel_Symbols(Rav[k], Rav[i], Rav[j], Metric)
                if c != 0 :
                    All_C[k,i,j] = smp.nsimplify(c)
                    #display(Latex('$' + latex(Gamma[Rav[i],Rav[j]]**Rav[k]) + ' = ' + latex(c) + '$'))
    
    return All_C

In [8]:
# Compute Riemann Tensor component
def Rie(alpha, beta, mu, nu, Metric, All_C):
    G1 = 0
    G2 = 0
    
    for k in (t,r,theta,phi):
        All_C
        G1 += All_C[Var[alpha], Var[mu], Var[k]] * All_C[Var[k], Var[beta], Var[nu]]
        G2 += All_C[Var[alpha], Var[nu], Var[k]] * All_C[Var[k], Var[beta], Var[mu]]
        
    return simplify(smp.diff(All_C[Var[alpha], Var[beta], Var[nu]], mu) - smp.diff(All_C[Var[alpha], Var[beta], Var[mu]], nu) + G1 - G2)

In [9]:
# Compute all Riemann Tensor components
def All_Riemann(Metric):
    All_C = All_Christoffel_Symbols(Metric)
    All_R = np.zeros((4,4,4,4), dtype=object)
    
    for i in range(4):
        for j in range(4):
            for l in range(4):
                for m in range(4):
                    R = Rie(Rav[i], Rav[j], Rav[l], Rav[m], Metric, All_C)
                    if R != 0 :
                        All_R[i,j,l,m] = R
                        #display(Latex('$' + latex(Riemann[Rav[j],Rav[l],Rav[m]]**Rav[i]) + ' = ' + latex(R) + '$'))
    return All_R

In [10]:
# Compute Ricci tensor components
def Ricci(mu,nu,Metric,All_R):
    Ri = 0
    for alpha in (t,r,theta,phi):
        Ri += All_R[Var[alpha],Var[mu],Var[alpha],Var[nu]]
    return simplify(Ri)

In [11]:
# Compute all Ricci tensor components
def All_Ricci(Metric):
    All_R = All_Riemann(Metric)
    All_Ri = np.zeros((4,4), dtype=object)

    for mu in range(4):
        for nu in range(4):
            Ri = Ricci(Rav[mu],Rav[nu],Metric,All_R)
            if Ri != 0:
                All_Ri[mu,nu] = Ri
                display(Latex('$' + latex(Riemann[Rav[mu],Rav[nu]]) + ' = ' + latex(Ri.subs([ (smp.diff(a(t),t,2), a_Dot(t)) , (smp.diff(a(t),t), a_dot(t)) ]) ) + '$'))
    return All_Ri


### Minkowski Metric

In [12]:
# Convention (-+++)
Mink_Metric = np.array([[-1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
Mink = smp.Matrix(Mink_Metric)

In [16]:
Mink

Matrix([
[-1, 0, 0, 0],
[ 0, 1, 0, 0],
[ 0, 0, 1, 0],
[ 0, 0, 0, 1]])

In [17]:
All_Ri_Mink = All_Ricci(Mink)

In [ ]:
smp.Matrix(All_Ri_Mink)

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

### Swcharzschil Metric

In [13]:
# Convention (-+++)
S_Metric = np.zeros((4,4))
S = smp.Matrix(S_Metric)

S[0,0] = -(1-2*G*M/r)
S[1,1] = 1/(1-2*G*M/r)
S[2,2] = r**2
S[3,3] = r**2*smp.sin(theta)**2

S = smp.nsimplify(S)

In [14]:
S

Matrix([
[2*G*M/r - 1,                0,    0,              0],
[          0, 1/(-2*G*M/r + 1),    0,              0],
[          0,                0, r**2,              0],
[          0,                0,    0, r**2*sin(θ)**2]])

In [ ]:
All_Ri_FLRW = All_Ricci(S)
smp.Matrix(All_Ri_FLRW)

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

### FLRW Metric

In [58]:
# Convention (+---)
FLRW_Metric = np.zeros((4,4))
FLRW = smp.Matrix(FLRW_Metric)

FLRW[0,0] = c**2
FLRW[1,1] = -a(t)**2/(1-k*r**2)
FLRW[2,2] = -a(t)**2*r**2
FLRW[3,3] = -a(t)**2*r**2*smp.sin(theta)**2

FLRW = smp.nsimplify(FLRW)

In [59]:
FLRW

Matrix([
[c**2,                      0,             0,                       0],
[   0, -a(t)**2/(-k*r**2 + 1),             0,                       0],
[   0,                      0, -r**2*a(t)**2,                       0],
[   0,                      0,             0, -r**2*a(t)**2*sin(θ)**2]])

In [75]:
All_Ri_FLRW = All_Ricci(FLRW)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>